In [5]:
import json
import pyhf
pyhf.set_backend('jax')

import pymc as pm

import numpy as np

import pytensor
from pytensor import tensor as pt
from pytensor.graph.basic import Apply
from pytensor.graph import Apply, Op

# import aesara
import aesara.tensor as at
from aesara.graph.op import Op
from aesara.link.jax.dispatch import jax_funcify

import jax
from jax import grad, jit, vmap, value_and_grad, random
import jax.numpy as jnp


# import sys
# sys.path.insert(1, '/Users/malinhorstmann/Documents/pyhf_pymc/src')
import MH_inference
import HMC_inference
import prepare_inference

## Model

In [2]:
### Simple pyhf model
dummy_model = pyhf.Model(
    {'channels': [{'name': 'singlechannel',
    'samples': [
    {'name': 'signal',
     'data': [6, 6, 3],
     'modifiers': [
         {'name': 'mu', 'type': 'normfactor', 'data': None}]},

    {'name': 'background',
     'data': [55, 55, 55],''
     'modifiers': [
        ## Staterror / Normal
        {"name": "my_staterror","type": "staterror","data": [2.0, 2.0, 2.4],},
        ## Lumi / Normal
        {'name': 'lumi', 'type': 'lumi', 'data': None},
        ## Correlated / Normal
        {'name': 'corr_bkg', 'type': 'histosys','data': {'hi_data': [65, 56, 67], 'lo_data': [40, 40, 43]}},
        {'name': 'corr_bkg1', 'type': 'histosys','data': {'hi_data': [65, 65, 66], 'lo_data': [40, 40, 40]}},
        {'name': 'corr_bkg2', 'type': 'histosys','data': {'hi_data': [66, 65, 60], 'lo_data': [40, 40, 39]}},
        ## Uncorrelated / Poisson
        # {'name': 'uncorr_bkg', 'type': 'shapesys','data': [7, 8, 7.17]},
        # {'name': 'uncorr_bkg1', 'type': 'shapesys','data': [7, 8, 6.7]},
        # {'name': 'uncorr_bkg2', 'type': 'shapesys','data': [7.27, 9, 7]},
        
         ]},    
                                 
    ]},
    ],
    "parameters": [
            {
                "name": "lumi",
                "auxdata": [1.0],
                "sigmas": [0.017],
                "bounds": [[0.915, 1.085]],
                "inits": [1.0],
            }
        ],}
)

dummy_nBins = len(dummy_model.expected_actualdata(dummy_model.config.suggested_init()))

### Observations
dummy_obs = dummy_model.expected_actualdata(dummy_model.config.suggested_init())

In [2]:
with open('SRee_SRmm_Srem.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
DL_model = workspace.model()
DL_obs = workspace.data(DL_model, include_auxdata=False)
DL_nBins = len(DL_model.expected_actualdata(DL_model.config.suggested_init()))


with open('ttbar_ljets_xsec_inclusive_pruned.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
ttbar_model = workspace.model()
ttbar_obs = workspace.data(ttbar_model, include_auxdata=False)
ttbar_nBins = len(ttbar_model.expected_actualdata(ttbar_model.config.suggested_init()))

## Testing with DL_model

In [16]:
def processedData(p):
    # a = jnp.stack([jax.jit(model.expected_actualdata(p))[i] for i in range(nBins)])
    a = jnp.stack([DL_model.expected_actualdata(p)[i] for i in range(DL_nBins)])
    return a

In [54]:
### Appling the Op to model.expected_actualdata
op, grad_op = HMC_inference.make_op(
    processedData,
    (at.TensorType(dtype=np.float64, shape=(len(DL_model.config.par_map),)),),
    (at.TensorType(dtype=np.float64, shape=(DL_nBins,)),),
)

a = np.linspace(0.01, 1, len(DL_model.config.par_names)).tolist()
pars = at.as_tensor_variable(a)
grad_op(pars, at.constant([1.0, 1.0, 1.0])).eval()

NameError: name 'processedData' is not defined

In [20]:
unconstr_dict = {
    'uncon1': {'type': 'unconstrained', 'type2': 'normal', 'input': [[1], [0.1]]}
    }


prior_dict = prepare_inference.prepare_priors(DL_model, unconstr_dict)

    # dictionary with keys 'model', 'obs', 'priors', 'precision'
prepared_model = prepare_inference.prepare_model(model=DL_model, observations=DL_obs, precision=0.10, priors=prior_dict)

### Building the `sampling()` function:


In [21]:
unconstr_pars, norm_pars, poiss_pars = [], [], []
norm_mu, norm_sigma = [], []
poiss_alpha, poiss_beta = [], []
model = prepared_model['model']
obs = prepared_model['obs']
prior_dict = prepared_model['priors']
precision = prepared_model['precision']

with pm.Model():
    
    for key in prior_dict.keys():
        sub_dict = prior_dict[key]

    ## Unconstrained
        if sub_dict['type'] == 'unconstrained':
            unconstr_pars.extend(pm.Normal('Unconstrained', mu=sub_dict['input'][0], sigma=sub_dict['input'][1]))
        pass

    ## Normal and Poisson constraints            
        if sub_dict['type'] == 'normal':
            norm_mu.append(sub_dict['input'][0])
            norm_sigma.append(sub_dict['input'][1])
        
        if sub_dict['type'] == 'poisson':
            poiss_alpha.append(sub_dict['input'][0])
            poiss_beta.append(sub_dict['input'][1])

    if np.array(norm_mu, dtype=object).size != 0:
        norm_pars.extend(pm.Normal('Normals', mu=list(np.concatenate(norm_mu)), sigma=list(np.concatenate(norm_sigma))))

    if np.array(poiss_alpha, dtype=object).size != 0:
        poiss_pars.extend(pm.Gamma('Gammas', alpha=list(np.concatenate(poiss_alpha)), beta=list(np.concatenate(poiss_beta))))

    pars = []
    for i in [unconstr_pars, norm_pars, poiss_pars]:
        i = np.array(i)
        if i.size != 0:
            pars.append(i)
    pars = np.concatenate(pars)
    target = prepare_inference.get_target(model)
    final = pt.as_tensor_variable(pars[target.argsort()].tolist())

print(final.type)


TensorType(float64, (18,))


In [28]:
# with pm.Model():
    
#     mu = grad_op(final, at.constant([1.0, 1.0, 1.0])).eval()
#     main = pm.Normal("main", mu=mu)#, observed=obs)

## New Ansatz

In [69]:
with open('SRee_SRmm_Srem.json') as serialized:
    spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
model = workspace.model()
obs = workspace.data(model, include_auxdata=False)
nBins = len(model.expected_actualdata(model.config.suggested_init()))

In [15]:
import matplotlib.pyplot as plt
import numpy as np

import pytensor
import pytensor.tensor as pt
from pytensor.graph import Apply, Op
from pytensor.link.jax.dispatch import jax_funcify

import jax
import jax.numpy as jnp

import pymc as pm
import pymc.sampling.jax

In [66]:
def custom_op_jax(x):
    return jnp.exp(x)

jitted_custom_op_jax = jax.jit(custom_op_jax)

jitted_custom_op_jax(np.arange(3))


def vjp_custom_op_jax(x, gz):
    _, vjp_fn = jax.vjp(custom_op_jax, x)
    return vjp_fn(gz)[0]

jitted_vjp_custom_op_jax = jax.jit(vjp_custom_op_jax)


In [65]:
@jax.jit
def vjp_func(func,fwd_inputs, vector):
    _,back = jax.vjp(func,fwd_inputs)
    return back(vector)

vjp_func(custom_expData, np.arange(18), at.constant([1.0, 1.0, 1.0]))

TypeError: Argument '<CompiledFunction of <function custom_expData at 0x288db8700>>' of type <class 'jaxlib.xla_extension.CompiledFunction'> is not a valid JAX type.

In [68]:
@jax.jit
def custom_expData(parameters):
    nBins = 3
    expData = jnp.stack([model.expected_actualdata(parameters)[i] for i in range(nBins)])

    return expData

print(custom_expData(np.arange(18)))

@jax.jit
def vjp_custom_expData(parameters, vector):
    _, vjp_fn = jax.vjp(custom_expData, parameters)
    return vjp_fn(vector)


print(vjp_custom_expData(np.arange(18), at.constant([1.0, 1.0, 1.0])))

AttributeError: 'Model' object has no attribute 'expected_actualdata'

In [56]:
class CustomOp(Op):
    # def make_node(self, x):
    #     # Create a PyTensor node specifying the number and type of inputs and outputs

    #     # We convert the input into a PyTensor tensor variable
    #     inputs = [pt.as_tensor_variable(x)]
    #     # Output has the same type and shape as `x`
    #     outputs = [inputs[0].type()]
    #     return Apply(self, inputs, outputs)

    # def perform(self, node, inputs, outputs):
    #     # Evaluate the Op result for a specific numerical input

    #     # The inputs are always wrapped in a list
    #     (x,) = inputs
    #     result = jitted_custom_op_jax(x)
    #     # The results should be assigned inplace to the nested list
    #     # of outputs provided by PyTensor. If you have multiple
    #     # outputs and results, you should assign each at outputs[i][0]
    #     outputs[0][0] = np.asarray(result, dtype="float64")

    # def grad(self, inputs, output_gradients):
    #     # Create a PyTensor expression of the gradient
    #     (x,) = inputs
    #     (gz,) = output_gradients
    #     # We reference the VJP Op created below, which encapsulates
    #     # the gradient operation
    #     return [vjp_custom_op(x, gz)]
        __props__ = ("jax_vjp_func",)

        def __init__(self):
            self.jax_vjp_func = vjp_func
            self.itypes = itypes + otypes
            self.otypes = itypes
            super().__init__()

        def perform(self, node, inputs, outputs):

            results = self.jax_vjp_func(*(jnp.asarray(x) for x in inputs))

            if not isinstance(results, (list, tuple)):
                results = (results,)

            for i, r in enumerate(results):
                outputs[i][0] = np.asarray(r)


class VJPCustomOp(Op):
    def make_node(self, x, gz):
        # Make sure the two inputs are tensor variables
        inputs = [pt.as_tensor_variable(x), pt.as_tensor_variable(gz)]
        # Output has the shape type and shape as the first input
        outputs = [inputs[0].type()]
        return Apply(self, inputs, outputs)

    def perform(self, node, inputs, outputs):
        (x, gz) = inputs
        result = jitted_vjp_custom_op_jax(x, gz)
        outputs[0][0] = np.asarray(result, dtype="float64")

# Instantiate the Ops
custom_op = CustomOp()
vjp_custom_op = VJPCustomOp()

In [57]:
pytensor.gradient.verify_grad(custom_op, (np.arange(5, dtype="float64"),), rng=np.random.default_rng())

In [59]:
with pm.Model() as model:
    x = pm.Normal("x", shape=(3,))
    y = pm.Normal("y", mu=custom_op(x))  # HERE IS WHERE WE USE THE CUSTOM OP!
    z = pm.Normal("z", y)

    a = pm.sample(200)

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [x, y, z]


Sampling 4 chains for 1_000 tune and 200 draw iterations (4_000 + 800 draws total) took 1 seconds.
